In [1]:
%load_ext autoreload
%autoreload 2

from commander.network import NetworkManager
from commander.network.protocol import (
    PingPacket,
    SetPositionPacket,
    SetVelocityPacket,
    FindLimitsPacket,
    ObservationPacket,
    RequestDebugInfoPacket,
    CheckLimitPacket,
    DoJigglePacket,
    DebugPacket,
    InfoPacket,
    ErrorPacket,
    ExperimentStartPacket,
    ExperimentStopPacket
)

from commander.network.constants import SetOperation, CartID
from time import time, sleep
import random
import numpy as np

nm = NetworkManager(port="COM3")
nm.open()

initialised = False
while not initialised:
    try:
        nm.read_initial_output()
    except UnicodeDecodeError:
        pass
    else:
        initialised = True

#$   YV===================
CartPole Controller
START DEBUG INFORMATION
STEPPER_CURRENT: 1000
STEPPER_MICROSTEPS: 8
STEPPER_DISTANCE_PER_ROTATION: 40.84
END DEBUG INFORMATION

Setting up I2C.
Configuring buttons.
Setting up steppers.
Starting stepper library.
Initiating astepper 1.
Setting up stepper drivers.
Setting up stepper driver 1.
CS (Vsense=1): 28
Microstepping: 32
Step delay in micros: 0
Sending 3
Received 101C1
Sending 9C7D7
Received 80E0
Sending A0000
Received 8080
Sending C001C
Received 8080
Sending E0050
Received 8080
Sending E0080
Received 8080
Sending C001F
Received D4C81
Sending A8202
Received D4C81
Sending 901B4
Received D4C81
Sending 3
Received D4CC1
INFO: Stall Guard level reached!
INFO: Motor is standing still.
Stall Guard value:851
Sending 3
Received D4CC1
ERROR: Channel B seems to be unconnected!
INFO: Stall Guard level reached!
INFO: Motor is standing still.
Stall Guard value:851
Microstepping: 8
Sending 5
Received D4CC1
ERROR: Channel B seems to be unconnected!
I

In [2]:
print(nm.serial.read_all().decode("ASCII", errors="ignore"))

In [3]:
nm.dump_packets()

# PingPacket

In [4]:
## P
ping_pkt = PingPacket(timestamp=10)

nm.send_packet(ping_pkt)

pkt = nm.read_packet()
print(pkt)

nm.serial.write(b"a")
pkt = nm.read_packet()
pkt

<PongPacket: id_: b'P', read_time: 2021-08-18 17:53:26.611432, timestamp: 10>


<DebugPacket: id_: b'#', msg: Received unknown packet with ID: 97, read_time: 2021-08-18 17:53:26.615441>

# UnknownPacket

In [5]:
nm.serial.write(b"a")
pkt = nm.read_packet()
print(pkt)

<DebugPacket: id_: b'#', msg: Received unknown packet with ID: 97, read_time: 2021-08-18 17:53:26.656349>


# RequestDebugInfoPacket

In [6]:
request_dbg_pkt = RequestDebugInfoPacket()
nm.send_packet(request_dbg_pkt)

sleep(0.010)
nm.dump_packets()

<DebugPacket: id_: b'#', msg: STEPPER_CURRENT: 1000, read_time: 2021-08-18 17:53:26.724436>
<DebugPacket: id_: b'#', msg: STEPPER_MICROSTEPS: 8, read_time: 2021-08-18 17:53:26.724436>
<DebugPacket: id_: b'#', msg: STEPPER_DISTANCE_PER_ROTATION: 40.840694, read_time: 2021-08-18 17:53:26.724436>
<DebugPacket: id_: b'#', msg: track_length_steps: 0, read_time: 2021-08-18 17:53:26.724436>
<DebugPacket: id_: b'#', msg: track_length_distance: 0.000000, read_time: 2021-08-18 17:53:26.724436>


# FindLimitsPacket

In [7]:
PKT = FindLimitsPacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

<DebugPacket: id_: b'#', msg: experiment_done: 0, read_time: 2021-08-18 17:53:26.764350>
Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [fast], read_time: 2021-08-18 17:53:28.643430>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [slow], read_time: 2021-08-18 17:53:29.049349>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [slow], read_time: 2021-08-18 17:53:29.585347>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT SET, read_time: 2021-08-18 17:53:29.601528>
<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT HIT [fast], read_time: 2021-08-18 17:53:34.586812>
<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT RETRACTED, read_time: 2021-08-18 17:53:34.742819>
<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT HIT [slow], read_time: 2021-08-18 17:53:36.425813>
<InfoPacket: id_: b'~', msg: LimitFinder: NOW DONE, read_time: 2021-08-18 17:53:37.265732>
<FindLimitsPacket: id_: b'|', read_ti

# CheckLimitPacket

In [8]:
PKT = CheckLimitPacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT HIT [fast], read_time: 2021-08-18 17:53:39.739858>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT RETRACTED, read_time: 2021-08-18 17:53:39.953129>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT HIT [slow], read_time: 2021-08-18 17:53:41.501224>
<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT GET, read_time: 2021-08-18 17:53:41.505276>
<InfoPacket: id_: b'~', msg: LimitChecker: New limit was 89, read_time: 2021-08-18 17:53:41.510130>
<InfoPacket: id_: b'~', msg: LimitChecker: NOW DONE, read_time: 2021-08-18 17:53:42.341130>
<CheckLimitPacket: id_: b'/', read_time: 2021-08-18 17:53:42.345245>


# DoJigglePacket

In [9]:
PKT = DoJigglePacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

Sent packet: <DoJigglePacket: id_: b'\xa7', read_time: None>
<DoJigglePacket: id_: b'\xa7', read_time: 2021-08-18 17:53:44.388804>


# SetPositionPacket

In [10]:
nm.dump_packets()

pkt = SetPositionPacket(operation=SetOperation.ADD, cart_id=CartID.ONE, value=100)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")
sleep(1)

nm.dump_packets()

Sent packet: <SetPositionPacket: cart_id: 1, id_: b'x', operation: +, value: 100>


# SetVelocityPacket

In [ ]:
nm.dump_packets()

pkt = SetVelocityPacket(operation=SetOperation.ADD, cart_id=CartID.ONE, value=100)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")
sleep(1)

nm.dump_packets()